In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [16]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
MAPBOX_TOKEN = user_secrets.get_secret("mapbox_token")
MAPBOX_STYLE = user_secrets.get_secret("mapbox_style")

ModuleNotFoundError: No module named 'kaggle_secrets'

In [5]:
def to_hour_str(number):
    string = '{:04d}'.format(number)
    string = string[:2] + ':' + string[2:]
    return string



In [6]:
fires = pd.read_csv("archive/fire_archive_M6_96619.csv")
print(fires.shape)


(36011, 15)


In [7]:
fires['acq_datestring'] = fires.apply(lambda r: r['acq_date'] + " " + to_hour_str(r['acq_time']), axis=1)

In [8]:
fires.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,acq_datestring
0,-11.8070,142.0583,313.0,1.0,1.0,2019-08-01,56,Terra,MODIS,48,6.3,297.3,6.6,D,0,2019-08-01 00:56
1,-11.7924,142.0850,319.3,1.0,1.0,2019-08-01,56,Terra,MODIS,71,6.3,297.3,11.3,D,0,2019-08-01 00:56
2,-12.8398,132.8744,311.6,3.1,1.7,2019-08-01,57,Terra,MODIS,42,6.3,298.7,23.1,D,0,2019-08-01 00:57
3,-14.4306,143.3035,310.1,1.1,1.1,2019-08-01,57,Terra,MODIS,33,6.3,296.1,6.5,D,0,2019-08-01 00:57
4,-12.4953,131.4897,310.3,4.0,1.9,2019-08-01,57,Terra,MODIS,36,6.3,298.8,27.6,D,0,2019-08-01 00:57


In [9]:
times = fires.groupby(['acq_date'])['acq_date'].count().index.tolist()
frames_data = [fires.loc[fires['acq_date'] == t] for t in times]

In [10]:
frames= [go.Frame(data=[go.Densitymapbox(lat=f['latitude'], lon=f['longitude'], z=f['brightness'], radius=10)], name=str(f.iloc[0]['acq_date'])) for f in frames_data]

In [11]:
buttons=[
         dict(label="Play",method="animate",args=[None, {'fromcurrent':True, "transition": {"duration": 30, "easing": "quadratic-in-out"}}]),
         dict(label="Pause",method="animate",args=[[None], {"frame": {"duration": 0, "redraw": False},"mode": "immediate", "transition": {"duration": 0}}])
]


sliders_dict = {
    'active':0,
    'currentvalue': dict(font=dict(size=16), prefix='Time: ', visible=True),
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    'x': 0,
    'steps': []
}

for i,t in enumerate(times):
    slider_step = {"args": [
                        [t],
                        {"frame": {"duration": 300, "redraw": False},
                         #"mode": "immediate",
                         "transition": {"duration": 30, "easing": "quadratic-in-out"}}
                    ],
            "label": t,
            "method": "animate",
            "value": t
    }
    sliders_dict['steps'].append(slider_step)

In [12]:
fig = go.Figure(data = [go.Densitymapbox(lat=fires['latitude'], lon=fires['longitude'], z=fires['brightness'], radius=1, colorscale='Hot', zmax=400, zmin=0)],
               layout=go.Layout(updatemenus=[dict(type="buttons", buttons=buttons,showactive=True)] ), 
               frames=frames
)

fig.update_layout(mapbox_style=MAPBOX_STYLE, 
                  mapbox_accesstoken=MAPBOX_TOKEN,
                  mapbox_center_lon=135,
                  mapbox_center_lat=-25.34,
                  mapbox_zoom=3.5)

"""fig.update_layout(mapbox_style="stamen-terrain", 
                  mapbox_center_lon=135,
                  mapbox_center_lat=-25.34,
                  mapbox_zoom=3.5)"""


fig.update_layout(sliders=[sliders_dict],
                 title='Australia fires over time')

fig.update_layout(height=850)
fig.show()


NameError: name 'MAPBOX_STYLE' is not defined